In [1]:
from src.util.data import get_data
from src.card.card import Card
from src.card.symbol import SymbolMapper
from src.model.embedding import get_embeddings, EmbeddingLookup
import openai
import os 
import numpy as np
from tqdm import tqdm

In [2]:
res = get_data('https://data.scryfall.io/oracle-cards/oracle-cards-20230416210326.json')
all_cards = [Card(data, processed=False) for data in tqdm(res) if data['object']=='card' and data['lang']=='en']
sm = SymbolMapper()

100%|██████████| 28578/28578 [00:48<00:00, 593.62it/s]


In [3]:
#check if embeddings lookup exists
if os.path.exists('bin/embeddings.json'):
    model_ins = [sm.map(str(card)) for card in tqdm(all_cards)]
    el = EmbeddingLookup()
    el.load('bin/embeddings.json')
else:
    model_ins = [sm.map(str(card)) for card in tqdm(all_cards)]
    embs = get_embeddings(model_ins,1000)
    el = EmbeddingLookup([card.id for card in all_cards], embs)
    el.save('bin/embeddings.json')

100%|██████████| 28568/28568 [00:00<00:00, 32152.77it/s]


In [4]:
lookup = "A deck that features Phyrexian creatures exclusively. Plenty of sacrifice and infect."

similar = el.get_similar(lookup, top_n = 75)
joinedStrings = '\n'.join([sc for sc,c in zip(model_ins,all_cards) if c.id in similar])

In [5]:
chatPrompt = """You are an assistant that helps build magic the gathering decks. 
You will be prompted with a desired type of deck and a list of cards suitable for the deck. 
The deck should be 60 cards, with no more than 4 copies of any card.
You should include appropriate basic lands (even if they are not in the list of cards.
You should include 24 lands total.
You should include 36 non-land cards total.
You should respond only with the card names.
You should carefully consider if you have the correct number of cards.
At the end of the prompt you should describe why the cards are good for the deck.
"""

prompt = f"Deck Description: {lookup} \nCards: {joinedStrings}"

In [6]:
openai.api_key = os.getenv('OPENAI_API_KEY')
completion = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": chatPrompt},
    {"role": "user", "content": prompt}
  ],
  max_tokens=1024,
)

In [7]:
print(completion['choices'][0]['message']['content'])

4x Phyrexian Juggernaut
4x Plague Stinger
4x Phyrexian War Beast
4x Pestilent Syphoner
4x Blight Pile
3x Phyrexian Reaper
3x Reaper of Sheoldred
2x Phyrexian Devourer
4x Plague Myr

4x Phyrexian Tower
20x Swamp

This deck focuses on Phyrexian creatures and has plenty of sacrifice and infect mechanics. Key creatures such as Phyrexian Juggernaut and Plague Stinger deal infect damage to opponents, which can be game-ending if they receive a total of 10 poison counters. The Phyrexian War Beast and Pestilent Syphoner will wreak havoc on your opponent's life total and creatures, and cards like Blight Pile, Reaper of Sheoldred, and Phyrexian Reaper provide utility and consistent damage output.

Phyrexian Tower serves as an efficient land that provides a sacrifice outlet and mana ramp, which is much needed in a deck with a higher average mana cost. A solid amount of Swamp cards will ensure you hit your land drops consistently.
